In [ ]:
import sys
import re
import pandas as pd
import numpy as np
import nltk
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from collections import Counter
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.losses import MeanSquaredLogarithmicError

In [ ]:
nltk.download('stopwords')
from nltk import word_tokenize

y = pd.read_csv('novacontent.csv', header=0)
#to drop first column
y=y.iloc[:, 1:]
y.head(5)

In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(y['Content'])
vectorizer.get_feature_names_out()

In [ ]:
X.toarray()

In [ ]:
X.toarray()[0]

In [ ]:
#vectorizer.inverse_transform(X.toarray())

In [ ]:
y['Content']=y['Content'].str.lower()
y['Content']=y['Content'].apply(word_tokenize)
#y.to_csv('content.csv')
y.to_csv('tokens.csv')

In [ ]:
y.head(5)

In [ ]:
all_words=[]
for i in y['Content']:
    for j in i:
        all_words.append(j)


In [ ]:
Counter(all_words)

In [ ]:
#without applying autoencoders for feature extraction(here it is words)

if_model=IsolationForest(n_estimators=100, contamination=0.03)
if_model.fit(X.toarray())
print(if_model)
score_pred = if_model.decision_function(X.toarray())
#print(score_pred)
pred = if_model.predict(X.toarray())
#print(pred)
print(type(pred))

In [ ]:
#shows that there are 740 positive logs and 21 negative logs

arr = np.array([pred])
list1 = arr.tolist()
print(list1)
print(type(list1))
Counter(pred)

In [ ]:
Y = X.toarray()

In [ ]:
X.toarray()[10]

In [ ]:
#Shuffle the data to randomly pick the rows for training and testing

import random
random.shuffle(Y)

In [ ]:
#Splitting the data to train and test

split_len = 0.7 * len(Y)
split_len
X_train = Y[:int(split_len)]
X_test = Y[int(split_len):]


In [ ]:
X_train = pd.DataFrame(X_train)
X_train.to_csv('X_train.csv')

In [ ]:
X_test = pd.DataFrame(X_test)
X_test.to_csv('X_test.csv')

In [ ]:
len(X_train)

In [ ]:
len(X_test)

In [ ]:
#Scaling the data

standard_scaler = MinMaxScaler()
x_train_scaled = standard_scaler.fit_transform(X_train)
x_test_scaled = standard_scaler.transform(X_test)

In [ ]:
class AutoEncoders(Model):

  def __init__(self, output_units):

    super().__init__()
    self.encoder = Sequential(
        [
          Dense(32, activation="relu"),
          Dense(16, activation="relu"),
          Dense(7, activation="relu")
        ]
    )

    self.decoder = Sequential(
        [
          Dense(16, activation="relu"),
          Dense(32, activation="relu"),
          Dense(output_units, activation="sigmoid")
        ]
    )

def call(self, inputs):

  encoded = self.encoder(inputs)
  decoded = self.decoder(encoded)
  return decoded
  
auto_encoder = AutoEncoders(len(x_train_scaled))

auto_encoder.compile(
    loss='mae',
    metrics=['mae'],
    optimizer='adam'
)

history = auto_encoder.fit(
    x_train_scaled, 
    x_train_scaled, 
    epochs=15, 
    batch_size=32, 
    validation_data=(x_test_scaled, x_test_scaled)
)